In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import sys
import pprint
import warnings
warnings.filterwarnings("ignore")
import math
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='653cfed9a26e46dca3e3e211ded03c0f',\
                                                      client_secret='a14b7886714c41c383458b26434d60cc')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
import spotipy
import json
import time
import copy
import scipy.sparse as ssp
from collections import Counter, defaultdict
import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from scipy.stats import pearsonr
from __future__ import print_function    
import time
from scipy import stats as sss
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.neural_network import MLPRegressor
import pickle
from scipy import io

In [3]:
validation_df = pd.read_csv('/Users/osx/Desktop/Warner Music Dissertation/Code/validation_df.csv')

In [46]:
validation_df = validation_df.drop('Unnamed: 0',axis=1)

# Generate user listen histroy by dictionary

In [47]:
music = validation_df[['customer_id','track_name']]
music.sort_values('customer_id',inplace=True)

# track_set = music['track_name'].unique() ######
users_, tracks = music.T.values

In [48]:
user, index = np.unique(users_,True)
user_history = np.split(tracks,index[1:])

# index_to_user_dict = dict(enumerate(user,start=0))
# index_to_track_id = dict(enumerate(track_set,start=0))

# inv_track_id_map = {v: k for k, v in index_to_track_id.items()}

In [49]:
v={u:defaultdict(int) for u in user}

for u,history in zip(user,user_history):
        for song in history:
            v[u][song]+=1

In [14]:
t = np.load('trainset_Dict.npy').item()
index_to_user_dict = np.load('train_user_map.npy').item()
index_to_track_id = np.load('train_track_map.npy').item()
inv_track_id_map = np.load('inv_track_map.npy').item()

In [51]:
v_user = v.keys()
v_f = {}
non_ative = []
n=0

for i in tqdm.tqdm(range(15000)):
    if index_to_user_dict[i] in v_user:
        v_f[index_to_user_dict[i]] = v[index_to_user_dict[i]]
        
    else:
        n+=1
        non_ative.append(i)
        v_f[index_to_user_dict[i]] = t[index_to_user_dict[i]]

100%|██████████| 15000/15000 [00:03<00:00, 4611.73it/s]


In [52]:
n

3289

In [53]:
track_base = []
for i in tqdm.tqdm(range(15000)):
    track_base.extend(list(t[index_to_user_dict[i]].keys()))
track_base = set(track_base)

col_num = len(track_base)

100%|██████████| 15000/15000 [00:00<00:00, 351395.54it/s]


In [54]:
val_mat = ssp.dok_matrix((15000,col_num))

for i in tqdm.tqdm(range(15000)):
    for j in v_f[index_to_user_dict[i]].keys():
        if j in track_base:
            k = inv_track_id_map[j]
            val_mat[i,k] = v_f[index_to_user_dict[i]][index_to_track_id[k]]

100%|██████████| 15000/15000 [00:00<00:00, 17288.14it/s]


In [56]:
# io.mmwrite("val_mat.mtx", val_mat)

In [65]:
# np.save('v_f',v_f)

In [62]:
# with open("non_active.txt", "wb") as fp:   #Pickling
#     pickle.dump(non_ative, fp)

In [68]:
np.save('track_base',track_base)

### Evaluate 1: MAE and RMSE of predict with ground truth

In [16]:
user_cosine = io.mmread("user_cosine.mtx")
user_item = io.mmread("user_item.mtx")
user_cosine = user_cosine.tocsr()
user_item = user_item.tocsr()

In [18]:
col_num=18419

In [19]:
mat_col = np.array(range(col_num))

In [23]:
### Calculate the MAE and RMSE with user-based cosine similarity
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = user_cosine[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_1 = MAE_numerator/value_denominator
RMSE_1 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of user-based cosine similarity is',MAE_1)
print('')
print('RMSE of user-based cosine similarity is',RMSE_1)

100%|██████████| 15000/15000 [04:15<00:00, 58.34it/s]

MAE of user-based cosine similarity is 0.0365024852249

RMSE of user-based cosine similarity is 0.5980833210926307


In [20]:
user_Pearson = io.mmread("user_Pearson.mtx")
user_Pearson = user_Pearson.tocsr()

In [25]:
# user_Pearson.todense()

In [26]:
### Calculate the MAE and RMSE with user-based Pearson correlation
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = user_Pearson[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_2 = MAE_numerator/value_denominator
RMSE_2 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of user-based Pearson Correlation is',MAE_2)
print('')
print('RMSE of user-based Pearson Correlation is',RMSE_2)

100%|██████████| 15000/15000 [04:32<00:00, 55.10it/s]

MAE of user-based Pearson Correlation is 0.0074187838705

RMSE of user-based Pearson Correlation is 0.08569266225468979


In [21]:
Item_cosine = io.mmread("Item_cosine.mtx")
Item_cosine = Item_cosine.tocsr()

In [28]:
### Calculate the MAE and RMSE with item-based cosine similarity
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Item_cosine[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_3 = MAE_numerator/value_denominator
RMSE_3 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Item cosine similarity is',MAE_3)
print('')
print('RMSE of Item cosine similarity is',RMSE_3)

100%|██████████| 15000/15000 [04:16<00:00, 57.77it/s]

MAE of Item cosine similarity is 0.00650860471822

RMSE of Item cosine similarity is 0.05248786069718772


In [22]:
Item_adjusted_cosine = io.mmread("Item_adjusted_cosine.mtx")
Item_adjusted_cosine = Item_adjusted_cosine.tocsr()

In [278]:
### Calculate the MAE and RMSE with item-based adjusted cosine similarity
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Item_adjusted_cosine[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_4 = MAE_numerator/value_denominator
RMSE_4 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Item adjusted cosine similarity is',MAE_4)
print('')
print('RMSE of Item adjusted cosine similarity is',RMSE_4)

100%|██████████| 15000/15000 [04:03<00:00, 59.44it/s]

MAE of Item adjusted cosine similarity is 0.00542927459719

RMSE of Item adjusted cosine similarity is 0.0570417848872202


In [23]:
Item_Pearson = io.mmread("Item_Pearson.mtx")
Item_Pearson = Item_Pearson.tocsr()

In [279]:
### Calculate the MAE and RMSE with item-based Pearson Correlation
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Item_Pearson[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_5 = MAE_numerator/value_denominator
RMSE_5 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Item Pearson Correlation is',MAE_5)
print('')
print('RMSE of Item Pearson Correlation is',RMSE_5)

100%|██████████| 15000/15000 [04:14<00:00, 59.02it/s]

MAE of Item Pearson Correlation is 0.00225218437626

RMSE of Item Pearson Correlation is 0.037870356850816016


In [131]:
### Calculate the MAE and RMSE with average of all similarity based results
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    avg_pred = (user_cosine[i,pred_col].toarray()+user_Pearson[i,pred_col].toarray()+\
               Item_cosine[i,pred_col].toarray()+Item_adjusted_cosine[i,pred_col].toarray()+
               Item_Pearson[i,pred_col].toarray())/5
    minus = avg_pred - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE = MAE_numerator/value_denominator
RMSE =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of average of five similarity pred is',MAE)
print('')
print('RMSE of average of five similarity pred is',RMSE)


100%|██████████| 15000/15000 [04:59<00:00, 50.01it/s]

MAE of average of five similarity pred is 0.00903180044483

RMSE of average of five similarity pred is 0.1292751168908239


In [24]:
Content_based = io.mmread("Content_based.mtx")
Content_based = Content_based.tocsr()

In [17]:
### Calculate the MAE and RMSE with content-based algorithm
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Content_based[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_6 = MAE_numerator/value_denominator
RMSE_6 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Content based cosine is',MAE_6)
print('')
print('RMSE of Content based cosine is',RMSE_6)

100%|██████████| 15000/15000 [04:30<00:00, 55.46it/s]

MAE of Content based cosine is 0.899731454529

RMSE of Content based cosine is 1.8404306538682036


In [25]:
### Calculate the MAE and RMSE with average of all similarity based results
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    avg_pred = (user_cosine[i,pred_col].toarray()+user_Pearson[i,pred_col].toarray()+\
               Item_cosine[i,pred_col].toarray()+Item_adjusted_cosine[i,pred_col].toarray()+
               Item_Pearson[i,pred_col].toarray()+Content_based[i,pred_col].toarray())/6
    minus = avg_pred - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE = MAE_numerator/value_denominator
RMSE =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of average of six similarity pred is',MAE)
print('')
print('RMSE of average of six similarity pred is',RMSE)

  0%|          | 0/15000 [00:00<?, ?it/s]


NameError: name 'val_mat' is not defined

### Evaluate 2: Recall and Precision metric

In [40]:
active_user = np.array(range(15000))
active_user = np.delete(active_user,non_ative)

In [18]:
# np.save('active_user.npy',active_user)

In [41]:
### filter the new tracks in validation dataset which not shown in training dataset
v_ft = {u:defaultdict(int) for u in v_f.keys()}

for i in tqdm.tqdm(range(15000)):
    for k in v_f[index_to_user_dict[i]].keys():
        if k in track_base:
            v_ft[index_to_user_dict[i]][k]=v_f[index_to_user_dict[i]][k]

100%|██████████| 15000/15000 [00:00<00:00, 22891.82it/s]


In [13]:
# np.save('validation_Dict',v_ft)

In [4]:
recommender_cosine_U = np.load('recommender_cosine_U.npy').item()

In [21]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_cosine_U[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_cos_u = suc_rec/Tnum_val
precision_cos_u = suc_rec/num_p
print('User_based cosine similarity recall is',recall_cos_u)
print('User_based cosine similarity precision',precision_cos_u)

User_based cosine similarity recall is 0.03119329948866448
User_based cosine similarity precision 0.02906668943728119


In [5]:
recommender_Pearson_U = np.load('recommender_Pearson_U.npy').item()

In [23]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_Pearson_U[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_P_u = suc_rec/Tnum_val
precision_P_u = suc_rec/num_p
print('User_based Pearson correlation recall is',recall_P_u)
print('User_based Pearson correlation precision',precision_P_u)

User_based Pearson correlation recall is 0.013122445613327713
User_based Pearson correlation precision 0.012227819998292203


In [6]:
recommender_cosine_T = np.load('recommender_cosine_T.npy').item()

In [25]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_cosine_T[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_cos_t = suc_rec/Tnum_val
precision_cos_t = suc_rec/num_p
print('Track based cosine similarity recall is',recall_cos_t)
print('Track based cosine similarity precision is',precision_cos_t)

Track based cosine similarity recall is 0.010794860986382713
Track based cosine similarity precision is 0.01005891896507557


In [7]:
recommender_adjusted_cosine_T = np.load('recommender_adjuested_cosine_T.npy').item()

In [27]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_adjusted_cosine_T[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_adcos_t = suc_rec/Tnum_val
precision_adcos_t = suc_rec/num_p
print('Track based adjusted cosine similarity recall is',recall_adcos_t)
print('Track based adjusted cosine similarity precision is',precision_adcos_t)

Track based adjusted cosine similarity recall is 0.006322966112567124
Track based adjusted cosine similarity precision is 0.005891896507556998


In [8]:
recommender_Pearson_T = np.load('recommender_Pearson_T.npy').item()

In [33]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_Pearson_T[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_P_t = suc_rec/Tnum_val
precision_P_t = suc_rec/num_p
print('Track based Pearson correlation recall is',recall_P_t)
print('Track based Pearson correlation precision is',precision_P_t)

Track based Pearson correlation recall is 0.009713542143943698
Track based Pearson correlation precision is 0.009051319272478866


In [34]:
### combine all five recommendation sets
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    rec_set = []
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set.extend(recommender_cosine_U[i])
    rec_set.extend(recommender_Pearson_U[i])
    rec_set.extend(recommender_cosine_T[i])
    rec_set.extend(recommender_adjusted_cosine_T[i])
    rec_set.extend(recommender_Pearson_T[i])
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall = suc_rec/Tnum_val
precision = suc_rec/num_p
print('Five recommendation sets recall is',recall)
print('Five recommendation sets precision is',precision)

Five recommendation sets recall is 0.05190330443707274
Five recommendation sets precision is 0.009672957048928357


In [9]:
recommender_conten_based = np.load('recommender_content_based.npy').item()

In [42]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_conten_based[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_content = suc_rec/Tnum_val
precision_content = suc_rec/num_p
print('content-based recall is',recall_content)
print('content-based precision is',precision_content)

content-based recall is 0.00047651338819346443
content-based precision is 0.00044402698317820853


In [12]:
# v_ft=np.load('v_f.npy').item()
# active_user=np.load('active_user.npy')

In [15]:
### combine all six recommendation sets
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    rec_set = []
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set.extend(recommender_cosine_U[i])
    rec_set.extend(recommender_Pearson_U[i])
    rec_set.extend(recommender_cosine_T[i])
    rec_set.extend(recommender_adjusted_cosine_T[i])
    rec_set.extend(recommender_Pearson_T[i])
    rec_set.extend(recommender_conten_based[i])
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall = suc_rec/Tnum_val
precision = suc_rec/num_p
print('Six recommendation sets recall is',recall)
print('Six recommendation sets precision is',precision)

Six recommendation sets recall is 0.041364222483012414
Six recommendation sets precision is 0.008126263057524265


##### Figure

In [296]:
visulazition = {'MAE':{'user_based cosine simialrity':MAE_1,'user_based Pearson correlation':MAE_2,\
         'track_based cosine similarity':MAE_3,'track_based adjusted cosine simialrity':MAE_4,\
         'track_based Pearson correlation':MAE_5,'average of five algorithms prediction':MAE},\
     'RMSE':{'user_based cosine simialrity':RMSE_1,'user_based Pearson correlation':RMSE_2,\
         'track_based cosine similarity':RMSE_3,'track_based adjusted cosine simialrity':RMSE_4,\
         'track_based Pearson correlation':RMSE_5,'average of five algorithms prediction':RMSE}}

In [298]:
pd.DataFrame.from_dict(visulazition)

,MAE,RMSE
average of five algorithms prediction,0.009032,0.129275
track_based Pearson correlation,0.002252,0.037870
track_based adjusted cosine simialrity,0.005429,0.057042
track_based cosine similarity,0.006509,0.052488
user_based Pearson correlation,0.007419,0.085693
user_based cosine simialrity,0.036502,0.598083


In [300]:
visulazition_1 = {'SRR':{'user_based cosine simialrity':suc_rate_1,\
                         'user_based Pearson correlation':suc_rate_2,\
                         'track_based cosine similarity':suc_rate_3,\
                         'track_based adjusted cosine simialrity':suc_rate_4,\
                         'track_based Pearson correlation':suc_rate_5,\
                        'add five together':suc_rate}}

In [301]:
pd.DataFrame.from_dict(visulazition_1)

,SRR
add five together,0.051903
track_based Pearson correlation,0.009714
track_based adjusted cosine simialrity,0.006323
track_based cosine similarity,0.010795
user_based Pearson correlation,0.013122
user_based cosine simialrity,0.031193


In [299]:
import plotly.plotly as py
import plotly.figure_factory as ff

data_matrix = [['algorithm name','MAE', 'RMSE'],
               ['user_based cosine simialrity', MAE_1, RMSE_1],
               ['user_based Pearson correlation', MAE_2, RMSE_2],
               ['track_based cosine similarity', MAE_3, RMSE_3],
               ['track_based adjusted cosine simialrity', MAE_4, RMSE_4],
               ['track_based Pearson correlation', MAE_5, RMSE_5],
              ['average of five algorithms prediction', MAE, RMSE]]

table = ff.create_table(data_matrix)
py.iplot(table, filename='MAE_and_RMSE_metrics')